In [ ]:
#Import Libraries
import requests  ## HTTP handling
import re        ## Regular expression function
import os        ## Create directory structures
import sys       ## 
import codecs
from bs4 import BeautifulSoup  # HTML parsing
from selenium import webdriver #Simulate Browser
from datetime import datetime         # Datetime object

#CHROME_DRIVER_PATH = "./chromedriver"
CHROME_DRIVER_PATH = "chromedriver.exe"
browser = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)

## Author Muhammad Valiallah

## Scraping from IMSDB

In [ ]:
#Link to IMSB script library
link_IMSDB = 'http://www.imsdb.com/all%20scripts/'
# Obtain links for all scripts
response_IMSDB = requests.get(link_IMSDB)
soup = BeautifulSoup(response_IMSDB.text, 'html.parser')

find_tables = soup.findAll('td', valign='top')
all_movies = find_tables[2].findAll('a') # Extract 3rd list of tables

# Obtain list of movies ( # Movie title,Movie link,file friendly movie title )
movies = [(movie_info.string, \
           movie_info["href"],
           re.split("[,.]",movie_info.string)[0].replace(' ', '_')) #remove
           for movie_info in all_movies]

## Create Database and IMSDB table

In [ ]:
import sqlite3
conn = sqlite3.connect('scriptanalyser.db')
c = conn.cursor()
c.execute("CREATE TABLE IMSDB (title TEXT,genres TEXT, writer TEXT, script TEXT)")

In [ ]:
#Create function to insert Movie into Database
def extract_script(film, browser):
    title, link_to_movie_page, movie_title = film
    # Interrogate the page with all the movie information (ratings, writer, genre, link to script)
    full_html_link = u'http://www.imsdb.com' + link_to_movie_page
    response_script = requests.get(full_html_link)
    soup = BeautifulSoup(response_script.text, 'html.parser')

    # Get all relevant information (genre, writer, script) from page
    list_links = soup.findAll('table', "script-details")[0].findAll('a')
    genres = []
    writer = []
    script = ''
    for link in list_links:
        href = link['href']
        if href[0:7]== "/writer":
            writer.append(link.get_text())
        if href[0:7]== "/genre/":
            genres.append(link.get_text())
        if href[0:9]== "/scripts/":
            script = href

    # If the link to the script points to a PDF, skip this movie,

    if script == '' or script[-5:] != '.html':
            pass
    else:

        # Parse the webpage which contains the script text
        full_script_url =  u'http://www.imsdb.com' + script
        browser.get(full_script_url)
        page_text = browser.page_source
        soup = BeautifulSoup(page_text, 'html.parser')

        if len(soup.findAll('td', "scrtext"))!=1:
            pass

            # Normal scraping:
        else:            
            # Append title to list
            print(str(title),str(genres),str(writer))
            IMSDB_titles.append(str(title))
            text = soup.findAll('td', "scrtext")[0].get_text()
            # Write the script to database table 
            c.execute("INSERT INTO IMSDB(title, genres, writer, script) values (?,?,?,?)",
            [str(title), str(genres), str(writer), text])
            conn.commit()

# Iterate through all movie scripts and insert into the database


In [ ]:
IMSDB_titles = [] # store titles in a list
for i,film in enumerate(movies):
    extract_script(film, browser)

## Obtain Film IDs from TMDB and import the python wrapper for it

In [ ]:
import tmdbsimple as tmdb
## Get an API Key by creating an account on TMDB
tmdb.API_KEY = '2d475c1fdd80442d338ca293bcd1'
tmdb.language = 'en' # for English movies
tmdb.debug = True
search = tmdb.Search()


#Get all the Movie Ids from TMDB
TMDB_titles = []
TMDB_ids = []
TMDB_popularity = []
for movie_name in IMSDB_titles:
    response = search.movie(query=movie_name)
    #Extract Id, TMDB_TITLE, Popularity 
    for result in search.results:
        TMDB_titles.append(result['title'])
        TMDB_ids.append(result['id'])
        TMDB_popularity.append(result['popularity'])
    

## Now that we have film IDs we can obtain the film details

In [ ]:
#Get movie attributes
movie_attributes = []
for id in TMDB_ids:
    try:
        #print(id[1])
        attribute = tmdb.Movies(id)
        response = attribute.info()
        response2 = attribute.releases()
        Rating = [c['certification'] for c in attribute.countries if c['iso_3166_1'] == 'US']
        #print(exec("column_attributes"))
        #print(attribute.title)
        #movie_attributes.append(column_attributes, Rating[0])
        c.execute("""INSERT INTO TMDB(
                     TMDB_title, budget, runtime, TMDB_genres,vote_average,
                     TMDB_release_date, popularity,TMDB_original_title
                     ,imdb_id,TMDB_Rating,'TMDB_revenue','TMDB_vote_count','TMDB_id') values (?,?,?,?,?,?,?,?,?,?,?,?,?)
                     """,
            [str(attribute.title), str(attribute.budget), str(attribute.runtime),str(attribute.genres), str(attribute.vote_average),\
             str(attribute.release_date), str(attribute.popularity), str(attribute.original_title),\
             str(attribute.imdb_id),str(Rating[0]),str(attribute.revenue), str(attribute.vote_count), str(id) \
            ])
        conn.commit()
        print(attribute.title, attribute.budget, attribute.runtime, attribute.vote_average, attribute.release_date, attribute.popularity, attribute.original_title, attribute.imdb_id,Rating[0])
    except:
        print(id)

## Join the  IMSDB table with the with the TMDB table and load into pandas dataframe

In [ ]:
query = """select  a.title as IMSDB_title, a.genres, a.script,
            b.TMDB_title, b.budget, b.runtime, b.TMDB_genres,b.vote_average,
                     b.TMDB_release_date, b.popularity,b.TMDB_original_title
                     ,b.imdb_id,b.TMDB_Rating,b.TMDB_revenue,b.TMDB_vote_count,b.TMDB_id
             from IMSDB a 
             inner join TMDB b
             on (trim(upper(a.title)) = trim(upper(b.TMDB_title)))
             """

In [ ]:
import pandas as pd
df = pd.read_sql_query(sql=query, con = conn)
df.head()

## Scraping Data from Daily Scripts

In [ ]:
#Link to IMSB script library
link_daily = 'https://www.dailyscript.com/movie.html'
# Obtain links for all scripts
response_daily = requests.get(link_daily)
soup = BeautifulSoup(response_daily.text, 'html.parser')

all_movies = find_tables[0].findAll('a') 

In [ ]:
daily_list = []
for i,movie in enumerate(all_movies):
    #if i % 2 ==0 and :
    if ('.txt' in movie["href"]) or ('.html' in movie["href"]) or ('.doc' in movie["href"]) :
        #print(i,movie.string,movie["href"],movie)
        daily_list.append((movie.string,movie["href"]))
        

## Repeating process for second page

In [ ]:
#Link to IMSB script library
link_daily = 'https://www.dailyscript.com/movie_n-z.html'
# Obtain links for all scripts
response_daily = requests.get(link_daily)
soup = BeautifulSoup(response_daily.text, 'html.parser')
find_tables = soup.findAll('td', valign='top',scope="col")
all_movies = find_tables[0].findAll('a')
for i,movie in enumerate(all_movies):
    #if i % 2 ==0 and :
    if ('.txt' in movie["href"]) or ('.html' in movie["href"]) or ('.doc' in movie["href"]) :
        #print(i,movie.string,movie["href"],movie)
        daily_list.append((movie.string,movie["href"]))

## Loading Data into Table 

In [ ]:
c.execute("CREATE TABLE DAILYDB (title TEXT, script TEXT)")        

In [ ]:
for i,film in enumerate(daily_list):
    title, link_to_movie_page = film
    full_script_url =  u'https://www.dailyscript.com/' + link_to_movie_page
    browser.get(full_script_url)
    page_text = browser.page_source
    soup = BeautifulSoup(page_text, 'html.parser')
    text = soup.get_text()
    c.execute("INSERT INTO DAILYDB(title, script) values (?,?)",
            [str(title), text])
    conn.commit()

In [ ]:
## Next we find the Film IDs from TMDB

In [ ]:
dailydb_titles = c.execute('select title from DAILYDB')
dailydb_list_titles =[]
for i in  dailydb_titles:
    print(i[0])
    dailydb_list_titles.append(i[0])
    
TMDB_titles = []
TMDB_ids = []
TMDB_popularity = []
for movie_name in dailydb_list_titles:
    response = search.movie(query=movie_name)
    #Extract Id, TMDB_TITLE, Popularity 
    for result in search.results:
        TMDB_titles.append(result['title'])
        TMDB_ids.append(result['id'])
        TMDB_popularity.append(result['popularity'])
        
# Create table to store Movie attributes in the database
columns = ('TMDB_title', 'budget', 'runtime', 'TMDB_genres', 'vote_average','TMDB_release_date','popularity','TMDB_original_title'
           ,'imdb_id', 'TMDB_Rating','TMDB_revenue','TMDB_vote_count','TMDB_id')
column_list = ", ".join("{field} TEXT".format(field=field) for field in columns)
column_list
c.execute("CREATE TABLE TMDB_daily({column})".format(column=column_list))        

In [ ]:
#Get movie attributes
movie_attributes = []
for id in TMDB_ids:
    try:
        #print(id[1])
        attribute = tmdb.Movies(id)
        response = attribute.info()
        response2 = attribute.releases()
        Rating = [c['certification'] for c in attribute.countries if c['iso_3166_1'] == 'US']
        #print(exec("column_attributes"))
        #print(attribute.title)
        #movie_attributes.append(column_attributes, Rating[0])
        c.execute("""INSERT INTO TMDB_daily(
                     TMDB_title, budget, runtime, TMDB_genres,vote_average,
                     TMDB_release_date, popularity,TMDB_original_title
                     ,imdb_id,TMDB_Rating,'TMDB_revenue','TMDB_vote_count','TMDB_id') values (?,?,?,?,?,?,?,?,?,?,?,?,?)
                     """,
            [str(attribute.title), str(attribute.budget), str(attribute.runtime),str(attribute.genres), str(attribute.vote_average),\
             str(attribute.release_date), str(attribute.popularity), str(attribute.original_title),\
             str(attribute.imdb_id),str(Rating[0]),str(attribute.revenue), str(attribute.vote_count), str(id) \
            ])
        conn.commit()
        print(attribute.title, attribute.budget, attribute.runtime, attribute.vote_average, attribute.release_date, attribute.popularity, attribute.original_title, attribute.imdb_id,Rating[0])
    except:
        print(id)

## Similarly we performed scraping on another site

In [ ]:
link_daily = 'http://www.moviescriptsandscreenplays.com/index.html#top'
# Obtain links for all scripts
response_daily = requests.get(link_daily)
soup = BeautifulSoup(response_daily.text, 'html.parser')
find_tables = soup.findAll('td', valign='top')

In [ ]:
def get_movies(link):
    response_daily = requests.get(link)
    soup = BeautifulSoup(response_daily.text, 'html.parser')
    all_movie = soup.findAll('a')
    return all_movie

all_movies = get_movies('http://www.moviescriptsandscreenplays.com/index.html#top')\
+get_movies('http://www.moviescriptsandscreenplays.com/movie-scripts.html')+get_movies('http://www.moviescriptsandscreenplays.com/movie-scripts2.html')

## Extracting only text readable documents

In [ ]:
movie_script_db =[]
for j in all_movies:
    #print(j)
    try:
        if ('script'in j['href'] and '.pdf' not in j['href'] and 'http://' in j['href']) and j.string !='host' and j.string !='SimplyScripts':
            print(j['href'],j.string)
            movie_script_db.append((j.string,j['href']))
    except:
        pass

    
c.execute("CREATE TABLE MOVIEDB (title TEXT, script TEXT)")

moviedb_titles =[]
for i,film in enumerate(movie_script_db):
    title, link_to_movie_page = film
    full_script_url = link_to_movie_page
    print(title,link_to_movie_page)
    try:
        browser.get(full_script_url)
        page_text = browser.page_source
        soup = BeautifulSoup(page_text, 'html.parser')
        text = soup.get_text()
        c.execute("INSERT INTO MOVIEDB(title, script) values (?,?)",
            [str(title), text])
        moviedb_titles.append(title)
        conn.commit()
    except:
        pass
    
c.execute("CREATE TABLE TMDB_moviedb({column})".format(column=column_list))


#Get movie attributes
movie_attributes = []
for id in TMDB_ids:
    try:
        #print(id[1])
        attribute = tmdb.Movies(id)
        response = attribute.info()
        response2 = attribute.releases()
        Rating = [c['certification'] for c in attribute.countries if c['iso_3166_1'] == 'US']
        #print(exec("column_attributes"))
        #print(attribute.title)
        #movie_attributes.append(column_attributes, Rating[0])
        c.execute("""INSERT INTO TMDB_moviedb(
                     TMDB_title, budget, runtime, TMDB_genres,vote_average,
                     TMDB_release_date, popularity,TMDB_original_title
                     ,imdb_id,TMDB_Rating,'TMDB_revenue','TMDB_vote_count','TMDB_id') values (?,?,?,?,?,?,?,?,?,?,?,?,?)
                     """,
            [str(attribute.title), str(attribute.budget), str(attribute.runtime),str(attribute.genres), str(attribute.vote_average),\
             str(attribute.release_date), str(attribute.popularity), str(attribute.original_title),\
             str(attribute.imdb_id),str(Rating[0]),str(attribute.revenue), str(attribute.vote_count), str(id) \
            ])
        conn.commit()
        print(attribute.title, attribute.budget, attribute.runtime, attribute.vote_average, attribute.release_date, attribute.popularity, attribute.original_title, attribute.imdb_id,Rating[0])
    except:
        print(id)

### We then consolidated the scripts from the three tables using SQL and loaded the data into a pandas dataframe

In [ ]:
c.execute("""CREATE TABLE TMDB_Combined as 
  select TMDB_title,budget,runtime,TMDB_genres,vote_average,TMDB_release_date,popularity,TMDB_original_title,imdb_id,TMDB_Rating,TMDB_revenue,TMDB_vote_count,TMDB_id from
  TMDB_daily
  union
  select TMDB_title,budget,runtime,TMDB_genres,vote_average,TMDB_release_date,popularity,TMDB_original_title,imdb_id,TMDB_Rating,TMDB_revenue,TMDB_vote_count,TMDB_id from
  TMDB_moviedb
  union
  select TMDB_title,budget,runtime,TMDB_genres,vote_average,TMDB_release_date,popularity,TMDB_original_title,imdb_id,TMDB_Rating,TMDB_revenue,TMDB_vote_count,TMDB_id from
  TMDB""")
print(c.fetchone())

In [ ]:
c.execute("""create view SCRIPTS_V as
            Select upper(trim(title)) as title,script from IMSDB
                union all
            Select upper(trim(title)) as title,script from DAILYDB
                    union all
            Select upper(trim(title)) as title,script from MOVIEDB""")


In [ ]:
c.execute(""" create table SCRIPTS_COMBINED as select title,
            script from (Select title,script
                            ,row_number() over (partition by title order by length(script) desc) as rec
             from SCRIPTS_V) where rec = 1""")

In [ ]:
query = """select  a.title as IMSDB_title, null as genres, a.script,
            b.TMDB_title, b.budget, b.runtime, b.TMDB_genres,b.vote_average,
                     b.TMDB_release_date, b.popularity,b.TMDB_original_title
                     ,b.imdb_id,b.TMDB_Rating,b.TMDB_revenue,b.TMDB_vote_count,b.TMDB_id
             from SCRIPTS_COMBINED a 
             inner join TMDB_COMBINED b
             on (trim(upper(a.title)) = trim(upper(b.TMDB_title)))
             """

In [ ]:
## Finally load data into Pandas dataframe

In [ ]:
df = pd.read_sql_query(sql=query, con = conn)
df